In [24]:
from ema_workbench.em_framework.optimization import (
    ArchiveLogger, 
    EpsilonProgress, 
    to_problem,
    epsilon_nondominated
)

from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
    Scenario,
    Policy,
    MultiprocessingEvaluator,
)

from ema_workbench.util import ema_logging
from ema_workbench.analysis import parcoords

import pandas as pd
import numpy as np
from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns

## Hard Coded Model -- To be replaced by problem_formulation

In [2]:
model_a4, planning_steps = get_model_for_problem_formulation('A4 Only')
problem_a4 = to_problem(model_a4, searchover="levers")

# Model with 13 outcomes, neccesary to apply the constraits
model_all, planning_steps = get_model_for_problem_formulation('All Dikes')
problem_all = to_problem(model_all, searchover="levers")

## Read Results

In [3]:
scenarios_df = pd.read_csv('./output/selected_scenarios.csv')
scenarios_df.head()

,Run ID,A0_ID_flood_wave_shape,A1_Bmax,A1_Brate,A1_pfail,A2_Bmax,A2_Brate,A2_pfail,A3_Bmax,A3_Brate,A3_pfail,A4_Bmax,A4_Brate,A4_pfail,A5_Bmax,A5_Brate,A5_pfail,discount_rate_0,discount_rate_1,discount_rate_2
0,81588,10,169.386292,1.0,0.511877,56.380416,1.0,0.388673,166.529650,1.5,0.614779,208.495509,10.0,0.527863,123.922757,10.0,0.552623,2.5,3.5,2.5
1,65779,101,32.415085,1.0,0.134757,145.712881,1.5,0.628312,186.606480,10.0,0.610541,69.736586,10.0,0.127815,206.451415,1.0,0.715698,3.5,1.5,3.5
2,8250,122,58.885207,1.0,0.092854,65.842956,1.0,0.032612,203.676012,1.0,0.558106,93.546587,1.0,0.365987,89.015196,1.5,0.303521,1.5,1.5,1.5
3,18777,68,128.130163,10.0,0.791207,339.425606,10.0,0.849394,31.308689,10.0,0.968231,164.774507,10.0,0.109501,307.969700,10.0,0.583670,2.5,1.5,1.5
4,Reference,4,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,3.5,3.5,3.5


In [4]:
scenarios = [row['Run ID'] for _, row in scenarios_df.iterrows()]
scenarios

['81588', '65779', '8250', '18777', 'Reference']

In [5]:
# Assumes seeds are sequential starting at 0 -- could be adapted to arbitrary
results = {}
convergences = {}
archives = {}

for scenario in scenarios:
    results[scenario] = []
    convergences[scenario] = []
    archives[scenario] = []
    for seed in range(5):
        # Results and Convergences
        fn_head = './output/POLICY_SEARCH__'
        fn_tail = f'__scen{scenario}__seed{seed}.csv'

        res = pd.read_csv(fn_head + 'results' + fn_tail, index_col=0)
        results[scenario].append(res)

        conv = pd.read_csv(fn_head + 'convergence' + fn_tail, index_col=0)
        convergences[scenario].append(conv)

        # Archives
        fn_head = './archives/POLICY_SEARCH__'
        fn_tail = f'__scen{scenario}__seed{seed}.tar.gz'
        arch = ArchiveLogger.load_archives(fn_head + 'archive' + fn_tail)
        for key, df in arch.items():
            if 'Unnamed: 0' in df.columns:
                arch[key] = arch[key].drop('Unnamed: 0', axis=1)
        archives[scenario].append(arch)


## Filtering Out Pareto-Dominated Policies
(within each scenario)

In [6]:
policy_sets = {}
epsilon = [100, 0.01, 100, 100, 0.01]
for scenario in scenarios:
    df = epsilon_nondominated(results[scenario], epsilon, problem_a4)
    # df = df.drop([o.name for o in model_a4.outcomes], axis=1)
    policy_sets[scenario] = df
    n_policies = df.shape[0]
    print(f"Scenario {scenario} has {n_policies} non-dominated policies")
    # TODO: Write pareto-nondominated policies to a file

Scenario 81588 has 65 non-dominated policies
Scenario 65779 has 84 non-dominated policies
Scenario 8250 has 147 non-dominated policies
Scenario 18777 has 50 non-dominated policies
Scenario Reference has 43 non-dominated policies


## Convergence Plots

In [8]:
policy_sets['18777'].head()

,EWS_DaysToThreat,rfr_0_t0,rfr_0_t1,rfr_0_t2,rfr_1_t0,rfr_1_t1,rfr_1_t2,rfr_2_t0,rfr_2_t1,rfr_2_t2,...,A4_DikeIncrease_t1,A4_DikeIncrease_t2,A5_DikeIncrease_t0,A5_DikeIncrease_t1,A5_DikeIncrease_t2,A4_Expected_Annual_Damage,A4_Expected_Number_of_Deaths,Total_Infrastructure_Costs,Total_Expected_Annual_Damage,Total_Expected_Number_of_Deaths
0,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0.000000e+00,0.000000,1.554910e+08,0.000000e+00,0.000000
1,3,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,4.191532e+06,0.000242,1.109498e+08,8.093174e+06,0.000519
2,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,3.522422e+05,0.000021,1.536437e+08,3.522422e+05,0.000021
3,3,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,3.772840e+06,0.000215,1.063765e+08,1.843323e+07,0.001536
4,3,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0.000000e+00,0.000000,1.080517e+08,1.466039e+07,0.001320


In [10]:
archives['81588'][0][100].head()

,EWS_DaysToThreat,rfr_0_t0,rfr_0_t1,rfr_0_t2,rfr_1_t0,rfr_1_t1,rfr_1_t2,rfr_2_t0,rfr_2_t1,rfr_2_t2,...,A4_DikeIncrease_t1,A4_DikeIncrease_t2,A5_DikeIncrease_t0,A5_DikeIncrease_t1,A5_DikeIncrease_t2,A4_Expected_Annual_Damage,A4_Expected_Number_of_Deaths,Total_Infrastructure_Costs,Total_Expected_Annual_Damage,Total_Expected_Number_of_Deaths
0,0,0,1,0,0,0,0,0,0,1,...,5,0,10,0,7,0.0,0.0,1.027071e+09,0.0,0.0


In [11]:
# for scenario in scenarios:
#     pols = policy_sets[scenario]
#     hv = HypervolumeMetric(pols, problem_a4)
#     gd = GenerationalDistanceMetric(pols, problem_a4, d=1)
#     ei = EpsilonIndicatorMetric(pols, problem_a4)
#     ig = InvertedGenerationalDistanceMetric(pols, problem_a4, d=1)
#     sm = SpacingMetric(problem_a4)

#     metrics_by_seed = []

#     for archive in archives[scenario]:
#         metrics = []
#         for nfe, a in archive.items():
#             scores = {
#                 "generational_distance": gd.calculate(a),
#                 "hypervolume": hv.calculate(a),
#                 "epsilon_indicator": ei.calculate(a),
#                 "inverted_gd": ig.calculate(a),
#                 "spacing": sm.calculate(a),
#                 "nfe": int(nfe),
#             }
#             metrics.append(scores)
#         metrics = pd.DataFrame.from_dict(metrics)

#         # sort metrics by number of function evaluations
#         metrics.sort_values(by="nfe", inplace=True)
#         # print(f'{scenario}.{seed} metrics:')
#         # print(metrics)
#         metrics_by_seed.append(metrics)
    
#     fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

#     ax1, ax2, ax3, ax4, ax5, ax6 = axes

#     for metrics, convergence in zip(metrics_by_seed, convergences[scenario]):
#         ax1.plot(metrics.nfe, metrics.hypervolume)
#         ax1.set_ylabel("hypervolume")

#         ax2.plot(convergence.nfe, convergence.epsilon_progress)
#         ax2.set_ylabel("$\epsilon$ progress")

#         ax3.plot(metrics.nfe, metrics.generational_distance)
#         ax3.set_ylabel("generational distance")

#         ax4.plot(metrics.nfe, metrics.epsilon_indicator)
#         ax4.set_ylabel("epsilon indicator")

#         ax5.plot(metrics.nfe, metrics.inverted_gd)
#         ax5.set_ylabel("inverted generational\ndistance")

#         ax6.plot(metrics.nfe, metrics.spacing)
#         ax6.set_ylabel("spacing")

#     ax6.set_xlabel("nfe")
#     ax1.set_title(f'Scenario {scenario} Convergence Graphs')

#     sns.despine(fig)
#     plt.savefig()
#     plt.show()

## Filtering Policy Sets According to Constraints

In [19]:
policies_to_evaluate = {}

for scenario in scenarios:
    policies_to_evaluate[scenario] = []

    policies = policy_sets[scenario]
    for idx, policy in policies.iterrows():
        policy_name = f's{scenario}_p{idx}'
        policies_to_evaluate[scenario].append(Policy(policy_name,
                                              **policy.to_dict()))

    # baseline policy, needed for for relative constraits
    zero_policy = {"EWS_DaysToThreat": 0}
    for i in range(5):
        dike = "A" + str(i+1)
        zero_policy.update({f"{dike}_DikeIncrease_t{n}": 0 for n in planning_steps})
        zero_policy.update({f"rfr_{i}_t{n}": 0 for n in planning_steps})

    p_ref = Policy("Base Case", **zero_policy)
    policies_to_evaluate[scenario].append(p_ref)
    

In [13]:
scenarios_df = scenarios_df.set_index('Run ID')

In [14]:
scenarios_df.head()

,A0_ID_flood_wave_shape,A1_Bmax,A1_Brate,A1_pfail,A2_Bmax,A2_Brate,A2_pfail,A3_Bmax,A3_Brate,A3_pfail,A4_Bmax,A4_Brate,A4_pfail,A5_Bmax,A5_Brate,A5_pfail,discount_rate_0,discount_rate_1,discount_rate_2
Run ID,,,,,,,,,,,,,,,,,,,
81588,10,169.386292,1.0,0.511877,56.380416,1.0,0.388673,166.529650,1.5,0.614779,208.495509,10.0,0.527863,123.922757,10.0,0.552623,2.5,3.5,2.5
65779,101,32.415085,1.0,0.134757,145.712881,1.5,0.628312,186.606480,10.0,0.610541,69.736586,10.0,0.127815,206.451415,1.0,0.715698,3.5,1.5,3.5
8250,122,58.885207,1.0,0.092854,65.842956,1.0,0.032612,203.676012,1.0,0.558106,93.546587,1.0,0.365987,89.015196,1.5,0.303521,1.5,1.5,1.5
18777,68,128.130163,10.0,0.791207,339.425606,10.0,0.849394,31.308689,10.0,0.968231,164.774507,10.0,0.109501,307.969700,10.0,0.583670,2.5,1.5,1.5
Reference,4,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,175.000000,1.5,0.500000,3.5,3.5,3.5


In [15]:
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [20]:
# Rerun experiment to calculate all outcomes
experiment_results = {}
for scenario in scenarios:
    scenario_dict = {}
    for col in scenarios_df:
        scenario_dict.update({col : scenarios_df.loc[scenario, col]})
    ema_scenario = Scenario(scenario, **scenario_dict)

    with MultiprocessingEvaluator(model_all) as evaluator:
        experiment_results[scenario] = evaluator.perform_experiments(ema_scenario,
                                                    policies_to_evaluate[scenario])

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1 scenarios * 66 policies * 1 model(s) = 66 experiments
100%|██████████████████████████████████████████| 66/66 [00:09<00:00,  7.13it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1 scenarios * 85 policies * 1 model(s) = 85 experiments
100%|██████████████████████████████████████████| 85/85 [00:12<00:00,  6.65it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1 scenarios * 148 policies * 1 model(s) = 148 experiments
100%|████████████████████████████████████████| 148/148 [00:32<00:00,  4.58it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1 scenarios * 51 policies * 1 model(s) = 51 experiment

In [18]:
# # Plot results of experimentation
# for scenario in scenarios:
#     experiments, outcomes = experiment_results[scenario]
#     outcomes = pd.DataFrame(outcomes)
#     # Drop baseline row
#     data = outcomes
#     limits = parcoords.get_limits(data)
#     limits.loc[0, list(outcomes.columns)] = 0

#     # 30 colors, as sns.color_palette() has less than 18 colors.
#     colors = sns.color_palette("husl", 150)
#     paraxes = parcoords.ParallelAxes(limits, fontsize=10)

#     for i, (index, row) in enumerate(data.iterrows()):
#         paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
#     paraxes.legend()

#     plt.show()

------------------
## Apply Constraints

In [21]:
RATIO=1.25 # 1.25 = 1/0.8

In [45]:
satisficing_policies = []
for scenario in scenarios:
    experiments, outcomes = experiment_results[scenario]
    outcomes = pd.DataFrame(outcomes)

    # Calculate ratios
    outcomes['Urban Damages Ratio'] = (outcomes['A4_Expected_Annual_Damage']
                                       / (outcomes['A3_Expected_Annual_Damage']
                                          + outcomes['A5_Expected_Annual_Damage']))

    outcomes['Urban Deaths Ratio'] = (outcomes['A4_Expected_Number_of_Deaths']
                                      / (outcomes['A3_Expected_Number_of_Deaths']
                                         + outcomes['A5_Expected_Number_of_Deaths']))

    outcomes['Industrialized Farmers Damages Ratio'] = (outcomes['A4_Expected_Annual_Damage']
                                                        / (outcomes['A1_Expected_Annual_Damage']
                                                           + outcomes['A2_Expected_Annual_Damage']))

    outcomes['Industrialized Farmers Deaths Ratio'] = (outcomes['A4_Expected_Number_of_Deaths']
                                                       / (outcomes['A1_Expected_Number_of_Deaths']
                                                          + outcomes['A2_Expected_Number_of_Deaths']))
    
    # Determine whether ratios have been exceeded
    base_case = outcomes.iloc[len(outcomes)-1, :]
    outcomes = outcomes.iloc[:len(outcomes)-1, :]

    outcomes['Urban Damages Ratio Exceeded'] = \
            np.where(outcomes['Urban Damages Ratio']
                     > base_case['Urban Damages Ratio'],
                     True,
                     False)
    outcomes['Urban Deaths Ratio Exceeded'] = \
            np.where(outcomes['Urban Deaths Ratio']
                     > base_case['Urban Deaths Ratio'],
                     True,
                     False)

    outcomes['Industrialized Farmers Damages Ratio Exceeded'] = \
            np.where(outcomes['Industrialized Farmers Damages Ratio']
                     > RATIO*base_case['Industrialized Farmers Damages Ratio'],
                     True,
                     False)
    outcomes['Industrialized Farmers Deaths Ratio Exceeded'] = \
            np.where(outcomes['Industrialized Farmers Deaths Ratio']
                     > RATIO*base_case['Industrialized Farmers Deaths Ratio'],
                     True,
                     False)
    
    # Keep rows that satisfy all constraints (all False)
    constraint_cols = [
        'Urban Damages Ratio Exceeded',
        'Urban Deaths Ratio Exceeded',
        'Industrialized Farmers Damages Ratio Exceeded',
        'Industrialized Farmers Deaths Ratio Exceeded'
        ]
    satisficing_policy_outcomes = outcomes[outcomes[constraint_cols].sum(axis=1) == 0]

    for idx, _ in satisficing_policy_outcomes.iterrows():
        satisficing_policies.append(policies_to_evaluate[scenario][idx])

In [52]:
columns = ['Policy Name'] + [key for key in satisficing_policies[0]]
policies_df = pd.DataFrame(columns=columns)

for i, policy in enumerate(satisficing_policies):
    row = []
    for col in columns:
        if col == 'Policy Name':
            row.append(policy.name)
        else:
            row.append(policy[col])
    policies_df.loc[i] = row

In [53]:
policies_df.head()

,Policy Name,EWS_DaysToThreat,rfr_0_t0,rfr_0_t1,rfr_0_t2,rfr_1_t0,rfr_1_t1,rfr_1_t2,rfr_2_t0,rfr_2_t1,...,A4_DikeIncrease_t1,A4_DikeIncrease_t2,A5_DikeIncrease_t0,A5_DikeIncrease_t1,A5_DikeIncrease_t2,A4_Expected_Annual_Damage,A4_Expected_Number_of_Deaths,Total_Infrastructure_Costs,Total_Expected_Annual_Damage,Total_Expected_Number_of_Deaths
0,s81588_p10,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.325785e+08,4.095309e+07,0.004889
1,s81588_p11,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.382731e+08,2.980485e+07,0.004907
2,s81588_p12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.357600e+07,1.515257e+08,0.018771
3,s81588_p13,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.481009e+08,6.192135e+06,0.000767
4,s81588_p15,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.003245e+08,7.994660e+07,0.009317
